In [1]:
%load_ext autoreload
%autoreload 2
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="MIG-ff70592b-6c77-5bde-832d-88d1e18cad50" 
import matplotlib.pyplot as plt
import numpy as np
import torch
import yaml
from hydra.utils import instantiate
from PIL import Image
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from br.models.load_models import get_data_and_models
from br.models.save_embeddings import get_pc_loss, save_embeddings, save_emissions, get_pc_loss_chamfer
from br.models.compute_features import compute_features
from br.models.utils import get_all_configs_per_dataset
from br.features.plot import collect_outputs, plot, plot_stratified_pc
from br.models.compute_features import get_embeddings
from br.models.utils import get_all_configs_per_dataset
from br.features.reconstruction import stratified_latent_walk
from br.features.archetype import AA_Fast
from pathlib import Path
import pandas as pd
from br.features.utils import normalize_intensities_and_get_colormap, normalize_intensities_and_get_colormap_apply
import os


device = "cuda:0"

# Load data and models

In [2]:
# Set paths

os.chdir('/allen/aics/modeling/ritvik/projects/benchmarking_representations/')
save_path = './test_var_punctate_embeddings/'

In [3]:
# Util function
def get_data_and_models(dataset_name, batch_size, results_path, debug=False):
    data_list = get_data(dataset_name, batch_size, results_path, debug)
    all_models, run_names, model_sizes = load_model_from_path(dataset_name, results_path) # default list of models in load_models.py
    return data_list, all_models, run_names, model_sizes

In [5]:
# Get datamodules, models, runs, model sizes

dataset_name = 'other_punctate'
batch_size = 2
debug=False
results_path = '/allen/aics/modeling/ritvik/projects/benchmarking_representations/configs/results/'
data_list, all_models, run_names, model_sizes = get_data_and_models(dataset_name, batch_size, results_path, debug)

# compute embeddings and emissions

In [13]:
# Compute embeddings and reconstructions for each model

splits_list = ["train", 'val', "test"]
meta_key = None
eval_scaled_img = [False] * 5
eval_scaled_img_params = [{}] * 5
loss_eval_list = None
sample_points_list = [True, True, False, False, False]
skew_scale = 100
save_embeddings(
    save_path,
    data_list,
    all_models,
    run_names,
    debug,
    splits_list,
    device,
    meta_key,
    loss_eval_list,
    sample_points_list,
    skew_scale,
    eval_scaled_img,
    eval_scaled_img_params,
)

Processing train


 49%|███████████████████████████████████████████████████▍                                                      | 12174/25085 [35:00<39:48,  5.41it/s]Exception ignored in: <function _ConnectionBase.__del__ at 0x7fbab610ab00>
Traceback (most recent call last):
  File "/home/ritvik.vasan/anaconda3/envs/replearn/lib/python3.10/multiprocessing/connection.py", line 132, in __del__
    self._close()
  File "/home/ritvik.vasan/anaconda3/envs/replearn/lib/python3.10/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor
Traceback (most recent call last):
  File "/home/ritvik.vasan/anaconda3/envs/replearn/lib/python3.10/multiprocessing/queues.py", line 239, in _feed
    reader_close()
  File "/home/ritvik.vasan/anaconda3/envs/replearn/lib/python3.10/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/home/ritvik.vasan/anaconda3/envs/replearn/lib/python3.10/multiprocessing/connection.py", line 361, in _close
  

Processing val


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5376/5376 [15:50<00:00,  5.66it/s]


Processing test


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 25085/25085 [22:00<00:00, 18.99it/s]


Processing val


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5376/5376 [04:45<00:00, 18.82it/s]


Processing test


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5376/5376 [04:49<00:00, 18.59it/s]


In [15]:
# Save emission stats for each model 

max_batches = 2
save_emissions(
    save_path,
    data_list,
    all_models,
    run_names,
    max_batches,
    debug,
    device,
    loss_eval_list,
    sample_points_list,
    skew_scale,
    eval_scaled_img,
    eval_scaled_img_params,
)

['classical_image',
 'so3_image',
 '2048_ed_dgcnn',
 '2048_int_ed_vndgcnn',
 '2048_int_ed_vndgcnn_structnorm']

# Compute benchmarking features

In [ ]:
# Compute multi-metric benchmarking features

keys = ['image', 'image', 'pcloud', 'pcloud', 'pcloud']
max_embed_dim = 256
DATA_LIST = get_all_configs_per_dataset(results_path)
data_config_list = DATA_LIST[dataset_name]["data_paths"]

evolve_params = {
    "modality_list_evolve": keys,
    "config_list_evolve": data_config_list,
    "num_evolve_samples": 40,
    "compute_evolve_dataloaders": False,
    "eval_meshed_img": [False] * 5,
    "skew_scale": 100,
    "eval_meshed_img_model_type": [None] * 5,
    "only_embedding": False,
    "fit_pca": False,
}

loss_eval = get_pc_loss_chamfer()
loss_eval_list = [loss_eval] * 5
use_sample_points_list = [True, True, False, False, False]

classification_params = {"class_labels": ["structure_name", 'cell_stage']}
rot_inv_params = {"squeeze_2d": False, "id": "cell_id", 'max_batches': 40}

regression_params = {"df_feat": None, "target_cols": None, "feature_df_path": None}

compactness_params = {
    "method": "mle",
    "num_PCs": None,
    "blobby_outlier_max_cc": None,
    "check_duplicates": True,
}

splits_list = ["train", "val", "test"]
compute_embeds = False

metric_list = [
    "Rotation Invariance Error",
    "Evolution Energy",
    "Reconstruction",
    "Classification",
    "Compactness",
]


compute_features(
    dataset=dataset_name,
    results_path=results_path,
    embeddings_path=save_path,
    save_folder=save_path,
    data_list=data_list,
    all_models=all_models,
    run_names=run_names,
    use_sample_points_list=use_sample_points_list,
    keys=keys,
    device=device,
    max_embed_dim=max_embed_dim,
    splits_list=splits_list,
    compute_embeds=compute_embeds,
    classification_params=classification_params,
    regression_params=regression_params,
    metric_list=metric_list,
    loss_eval_list=loss_eval_list,
    evolve_params=evolve_params,
    rot_inv_params=rot_inv_params,
    compactness_params=compactness_params,
)

Computing rotation invariance


  1%|▊                                                                                                             | 41/5376 [00:04<10:15,  8.66it/s]


Getting reconstruction
Computing compactness


  0%|                                                                                                                          | 0/5 [00:00<?, ?it/s]

(71673, 256)
Outlier column is cell stage


...

/home/ritvik.vasan/anaconda3/envs/replearn/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/home/ritvik.vasan/anaconda3/envs/replearn/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/home/ritvik.vasan/anaconda3/envs/replearn/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/home/ritvik.vasan/ana

Computing classification


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [42:41<00:00, 512.38s/it]


Computing evolution


 11%|███████████▎                                                                                               | 570/5376 [13:29<1:50:58,  1.39s/it]

# Polar plot viz

In [ ]:
# Holistic viz of features
model_order = ["Classical_image", "SO3_image", "Classical_pointcloud", "SO3_pointcloud"]
metric_list = ['reconstruction', 'emissions', 'classification_cell_stage_fine', 'classification_flag_comment',
               'compactness', 'evolution_energy', 
               'model_sizes', 'rotation_invariance_error']
norm = 'std'
title = 'pcna_comparison'
colors_list = None
unique_expressivity_metrics = ['Classification_cell_stage_fine', 'Classification_flag_comment']
df, df_non_agg = collect_outputs(save_path, norm, model_order, metric_list)
plot(save_path, df, model_order, title, colors_list, norm, unique_expressivity_metrics)

In [7]:
1

1